In [1]:
import numpy as np
import pandas as pd
import itertools as it
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import roc_curve, auc,precision_recall_curve, roc_curve
import matplotlib.pylab as plt

import plotly.graph_objects as go

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
Data = pd.read_excel('Data.xlsx')
Data = Data.set_index('Date')
Y = Data['Class']
X = Data.drop('Class', axis = 1)

### Parameters

In [3]:
step = 100
niter = 10
ts_prop = 0.1
va_prop = 0.2

Hyper_Parameters = [['l1','l2']] # Regresores
set_parameters = [i for i in it.product(*Hyper_Parameters)]

#### Error real y de entrenamiento deseados

In [4]:
Epsilon = 0.4
delta = 0.15

#### Garantia probable de aprendizaje ($\eta_{estimado}$) y Tamaño optimo de la muestra $\eta_{optimo}$

#### $\eta_{estimado}$

In [5]:
n_est = int(np.ceil((1/Epsilon)*(np.log(len(set_parameters)) + np.log(1/delta))))

#### $\eta_{optimo}$

In [6]:
n_opt = X.shape[1]+1

#### Division del conjunto: entrenamiento (70%), validacion (20%) y prueba (10%)

In [7]:
X1, X1_ts, Y1,Y1_ts = train_test_split(X, Y, train_size = 1-ts_prop, random_state=19)
X1_tr, X1_va, Y1_tr, Y1_va = train_test_split(X1, Y1, train_size = 1-va_prop, random_state=19)

#### Discretizacion para explorar el aumento de $\eta$

In [8]:
N = range(n_est, round(X1_tr.shape[0]*(1-va_prop)), 100)

### Exploracion de modelos partiendo de $\eta_{estimado}$, es claro que $\eta_{estimado}$ es tenido en cuenta ya que $\eta_{estimado} < \eta_{optimo}$

In [9]:
n_est_mod_results = {
    'Model': [None for i in range(niter)],
    'Accuracy': np.zeros((len(N),niter)),
    'Precision': np.zeros((len(N),niter)),
    'Recall': np.zeros((len(N),niter))
}

In [10]:
for j in range(niter):
    lrc = LogisticRegression()#penalty= set_parameters[j][0])
    regresors = np.random.randint(0, X1_tr.shape[1], np.random.randint(0, X1_tr.shape[1]))
    for i in range(len(N)):
        np.random.seed(seed = 19)
        ind1 = np.random.randint(0, X1_tr.shape[0], N[i])
        lrc.fit(X1_tr.iloc[ind1, regresors], Y1_tr[ind1])
        y_pred = lrc.predict(X1_va.iloc[:,regresors])
        n_est_mod_results['Accuracy'][i,j] = accuracy_score(Y1_va, y_pred)
        n_est_mod_results['Precision'][i,j] = precision_score(Y1_va, y_pred, average = 'macro')
        n_est_mod_results['Recall'][i,j] = recall_score(Y1_va, y_pred, average = 'macro')
    n_est_mod_results['Model'][j] = (lrc,regresors)
    
ind_bst = np.where(n_est_mod_results['Accuracy'] == np.amax(n_est_mod_results['Accuracy']))[1][0]
n_est_mod_results['Best'] = set_parameters[ind_bst]

#### Grid search

In [11]:
Hyper_Parameters = {
    'penalty': ['l1', 'l2']
}

clf = GridSearchCV(LogisticRegression(), Hyper_Parameters, cv = 5)
clf.fit(X1_tr.values, Y1_tr)
clf.best_params_

{'penalty': 'l1'}

In [15]:
fig1 = go.Figure(data=[go.Surface(z=n_est_mod_results['Accuracy'])])

fig1.update_layout(title='Accuracy Surface', autosize=True,
                   scene = dict(xaxis_title='Modelos',
                                yaxis_title='Tamaño de N',
                                zaxis_title='Accuracy'))

fig1.show()
fig1.write_html('Acc_LOG_Normal.html')

In [16]:
fig2 = go.Figure(data=[go.Surface(z=n_est_mod_results['Precision'])])

fig2.update_layout(title='Precision Surface', autosize=True,
                   scene = dict(xaxis_title='Modelos',
                                yaxis_title='Tamaño de N',
                                zaxis_title='Precision'))

fig2.show()
fig2.write_html('Pre_LOG_Normal.html')

In [17]:
fig3 = go.Figure(data=[go.Surface(z=n_est_mod_results['Recall'])])

fig3.update_layout(title='Recall Surface', autosize=True,
                   scene = dict(xaxis_title='Modelos',
                                yaxis_title='Tamaño de N',
                                zaxis_title='Recall'))

fig3.show()
fig3.write_html('Rec_LOG_Normal.html')